# CNN Architectures 

## COVIDcxr Dataset

In [1]:
from fastai.vision.all import *
path = Path('/home/jupyter/covidcxr') 

In [2]:
torch.cuda.empty_cache()

In [15]:
# fix result 
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
SEED = 42
seed_everything(SEED)

In [16]:
df = pd.read_csv(path/'covidcxr.csv')
df

path  \
0    auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg   
1    auntminnie-b-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg   
2    auntminnie-c-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg   
3    auntminnie-d-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg   
4                                                     nejmc2001573_f1a.jpeg   
..                                                                      ...   
955                                                        00000191_000.png   
956                                                        00000192_000.png   
957                                                        00000193_000.png   
958                                                        00000193_001.png   
959                                                        00000193_002.png   

        finding   age sex view patientid  
0      COVID-19  65.0   M   PA         2  
1      COVID-19  65.0   M   PA         2  
2      COVID-19  65.0   M   PA         2  
3      COVID-19  65.0   M   PA         2  
4      COVID-19  52.0   F   PA         4  
..          ...   ...  ..  ...       ...  
955  No Finding  84.0   M   PA       191  
956  No Finding  87.0   M   PA       192  
957  No Finding  54.0   M   AP       193  
958  No Finding  54.0   M   AP       193  
959  No Finding  55.0   M   AP       193  

[960 rows x 6 columns]

In [17]:
get_x=lambda x:path/f"{x[0]}"
get_y=lambda x:x[1]
splitter=RandomSplitter(seed=SEED)
bs=32

In [18]:
metrics=[accuracy,  
         #RocAuc(average='macro', multi_class='ovr'),
         MatthewsCorrCoef(sample_weight=None),
         Precision(average='macro'),
         Recall(average='macro'),     
         F1Score(average='macro')] 

In [19]:
item_tfms=Resize(480, method='squish', pad_mode='zeros', resamples=(2, 0))
batch_tfms=[*aug_transforms(mult=1.0, do_flip=False, flip_vert=False, 
                            max_rotate=20.0, max_zoom=1.2, max_lighting=0.3, max_warp=0.2, 
                            p_affine=0.75, p_lighting=0.75, 
                            xtra_tfms=None, size=None, mode='bilinear', pad_mode='reflection', 
                            align_corners=True, batch=False, min_scale=1.0),
                            Normalize.from_stats(*imagenet_stats)]

In [20]:
db = DataBlock(blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
               get_x=get_x,
               get_y=get_y,
               splitter=splitter,
               item_tfms = item_tfms,
               batch_tfms=batch_tfms)

In [21]:
dl = db.dataloaders(df, bs=bs)

## Vgg16

In [10]:
from torchvision.models import vgg16
arch = vgg16

In [11]:
learn = cnn_learner(dl, arch=arch, metrics=metrics)

In [12]:
learn.fine_tune(30)

## Vgg19

In [10]:
from torchvision.models import vgg19
arch = vgg19

In [11]:
learn = cnn_learner(dl, arch=arch, metrics=metrics)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /home/jupyter/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


In [12]:
learn.fine_tune(30)

## ResNet18

In [22]:
arch = resnet18

In [23]:
learn = cnn_learner(dl, arch=arch, metrics=metrics)

In [24]:
learn.fine_tune(30)

## ResNet34

In [34]:
arch = resnet34

In [35]:
learn = cnn_learner(dl, arch=arch, metrics=metrics)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /home/jupyter/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [36]:
learn.fine_tune(30)

## ResNet50

In [46]:
arch = resnet50

In [47]:
learn = cnn_learner(dl, arch=arch, metrics=metrics)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/jupyter/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [48]:
learn.fine_tune(30)

## Efficientnet-B0

In [22]:
from efficientnet_pytorch import EfficientNet
arch = EfficientNet.from_pretrained("efficientnet-b0")

Loaded pretrained weights for efficientnet-b0


In [23]:
learn = Learner(dl, model=arch, metrics=metrics)

In [24]:
learn.fine_tune(30)

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
